In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
# add token here

In [ ]:
from datasets import load_dataset

dataset = load_dataset("irlab-udc/metahate")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

available_metahate.tsv:   0%|          | 0.00/307M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1101165 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1101165
    })
})


In [ ]:
pd.set_option("display.max_colwidth", None)

# agar rows/columns bhi limit nahi chahiye
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
df = dataset["train"].to_pandas()

In [ ]:
df.head()

,label,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [ ]:
df['label'].value_counts()

,count
label,
0,867876
1,233289


In [ ]:
df.isnull().sum()

,0
label,0
text,1


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
df = df.fillna('')

In [ ]:
#seperate data and label
x = df.drop(columns = 'label',axis = 1)
y = df['label']

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)   # remove URLs
    text = re.sub(r"[^a-zA-Z]", "", text)  # remove special chars
    text = text.lower().strip()
    return text

df['clean_text'] = df['text'].apply(clean_text)


In [ ]:
# Downsample with balancing
min_count = df['label'].value_counts().min()

df_balanced = df.groupby('label').apply(lambda x: x.sample(n=min(100000, len(x)), random_state=42)).reset_index(drop=True)


/tmp/ipython-input-4231195420.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('label').apply(lambda x: x.sample(n=min(100000, len(x)), random_state=42)).reset_index(drop=True)


In [ ]:
df_balanced.isnull().sum()

,0
label,0
text,0
clean_text,0


In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = clean_text(str(text))  # ensure string type
    words = text.split()
    filtered = []
    for w in words:
        w = w.strip()
        if len(w) > 1 and w.isalpha() and w not in stop_words:  # avoid short/invalid tokens
            try:
                filtered.append(port_stem.stem(w))
            except RecursionError:
                continue  # skip problematic tokens
    return " ".join(filtered)

df_balanced['final_text'] = df_balanced['clean_text'].apply(preprocess)



In [ ]:
df_balanced.tail()

,label,text,clean_text,final_text
199995,1,"@Impeach_D_Trump I don't have any Russian contacts @ all, but can act evil &amp; corrupt, yet stupid at the same time,… https://t.co/tww7P2i4xf",impeachdtrumpidonthaveanyrussiancontactsallbutcanactevilampcorruptyetstupidatthesametime,impeachdtrumpidonthaveanyrussiancontactsallbutcanactevilampcorruptyetstupidatthesametim
199996,1,"NEWLINE_TOKENNEWLINE_TOKENsince when do criticisms of particular know-it-all wikipedian, in a TALK page a cause for blocking? some might call that disruptive, if they are direct descendants of HITLER, and ban everybody who does not agree with them, or use what THEY consider civil language.",newlinetokennewlinetokensincewhendocriticismsofparticularknowitallwikipedianinatalkpageacauseforblockingsomemightcallthatdisruptiveiftheyaredirectdescendantsofhitlerandbaneverybodywhodoesnotagreewiththemorusewhattheyconsidercivillanguage,newlinetokennewlinetokensincewhendocriticismsofparticularknowitallwikipedianinatalkpageacauseforblockingsomemightcallthatdisruptiveiftheyaredirectdescendantsofhitlerandbaneverybodywhodoesnotagreewiththemorusewhattheyconsidercivillanguag
199997,1,@user He is the poster child for the GOP,userheistheposterchildforthegop,userheistheposterchildforthegop
199998,1,"bitch you & y'all weak ass body oil selection can suck my clit backwards, hoe.",bitchyouyallweakassbodyoilselectioncansuckmyclitbackwardshoe,bitchyouyallweakassbodyoilselectioncansuckmyclitbackwardsho
199999,1,"You're a bigger person than me, I would have told her to start sucking nigger dick cause dat bitch would've been taking my happy meal ass to Wrestlemania. Woooo!",youreabiggerpersonthanmeiwouldhavetoldhertostartsuckingniggerdickcausedatbitchwouldvebeentakingmyhappymealasstowrestlemaniawoooo,youreabiggerpersonthanmeiwouldhavetoldhertostartsuckingniggerdickcausedatbitchwouldvebeentakingmyhappymealasstowrestlemaniawoooo


In [ ]:
# vectorizer = TfidfVectorizer(max_features=5000)  # limit features for speed
# X = vectorizer.fit_transform(df_balanced['final_text']).toarray()
# y = df_balanced['label']

In [ ]:
df.head()

,label,text,clean_text
0,0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,rtmayasolovelyasawomanyoushouldntcomplainaboutcleaningupyourhouseampasamanyoushouldalwaystakethetrashout
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,rtmleewboydatscoldtygadwnbadforcuffindathoeinthestplace
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,rturkindofbranddawgrtsbabylifeyoueverfuckabitchandshestarttocryyoubeconfusedasshit
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,rtcgandersonvivabasedshelooklikeatranny
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,rtshenikarobertstheshityouhearaboutmemightbetrueoritmightbefakerthanthebitchwhotoldittoya


In [ ]:
df['hate_type'] = None  # new empty column
df.loc[df['label'] == 0, 'hate_type'] = 'non_hateful'

In [ ]:
df.tail()

,label,text,clean_text,hate_type
1101160,0,":Jerome, I see you never got around to this…! I’m not surprised: I looked for an example of the ‘’in nomine’’ plainsong notaion and couldn’t find one online - to my astonishment too. \n :But maybe you could clear up a question for me: is the cantus firmus of the English ‘’in nomines’’ the plainchant (in the alto part), or one of Taverner’s other three parts? This is not clear to me in the article. If the ‘’in nomines’’ are based on the plainchant, then where does Taverner come into it? Your expertise would be most appreciated, and maybe the article requires some explanation to this effect.",jeromeiseeyounevergotaroundtothisimnotsurprisedilookedforanexampleoftheinnomineplainsongnotaionandcouldntfindoneonlinetomyastonishmenttoobutmaybeyoucouldclearupaquestionformeisthecantusfirmusoftheenglishinnominestheplainchantinthealtopartoroneoftavernersotherthreepartsthisisnotcleartomeinthearticleiftheinnominesarebasedontheplainchantthenwheredoestavernercomeintoityourexpertisewouldbemostappreciatedandmaybethearticlerequiressomeexplanationtothiseffect,non_hateful
1101161,0,==Lucky bastard== \n http://wikimediafoundation.org/wiki/Press_releases/English_Wikipedia_Announces_Thousandth_Featured_Article \n\n Heh you are famous now. I kida envy that. Congrats.,luckybastardhehyouarefamousnowikidaenvythatcongrats,non_hateful
1101162,0,==shame on you all!!!== \n\n You want to speak about gays and not about romanians...,shameonyouallyouwanttospeakaboutgaysandnotaboutromanians,non_hateful
1101163,1,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MOVIES. HE HAS SO MUCH BUTTSEX THAT HIS ASSHOLE IS NOW BIG ENOUGH TO BE CONSIDERED A COUNTRY.,melgibsonisanazibitchwhomakesshittymovieshehassomuchbuttsexthathisassholeisnowbigenoughtobeconsideredacountry,None
1101164,0,""" \n\n == Unicorn lair discovery == \n\n Supposedly, a 'unicorn lair' has been discovered in Pyongyang, North Korea. The lair is supposedly associated with King Dongmyeong of Goguryeo, who supposedly rode a unicorn. It should be added, but i can't quite find where to insert it. """,unicornlairdiscoverysupposedlyaunicornlairhasbeendiscoveredinpyongyangnorthkoreathelairissupposedlyassociatedwithkingdongmyeongofgoguryeowhosupposedlyrodeaunicornitshouldbeaddedbuticantquitefindwheretoinsertit,non_hateful


In [ ]:
# Python cell
import os, math, random, time
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm

# USER CONFIG - tweak these if needed
SAMPLE_SIZE = 6000            # how many hateful rows to label with zero-shot teacher
ZERO_SHOT_BATCH = 32          # batch size for teacher inference (increase if GPU)
FINETUNE_BATCH = 16           # batch size for student training (reduce if OOM)
FINETUNE_EPOCHS = 3
MAX_LEN = 128                 # tokenizer max length
ZERO_SHOT_MODEL = "facebook/bart-large-mnli"   # teacher model (stable)
STUDENT_MODEL = "distilbert-base-uncased"      # student model to fine-tune
OUTPUT_DIR = "./hate_subtype_model"

# candidate labels for hate types
CANDIDATE_LABELS = [
    "sarcasm-based hate",
    "humor-based hate",
    "metaphor-based hate",
    "political framing hate",
    "meme-language hate"
]

device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if device==0 else "CPU")



Using device: GPU


In [ ]:


df.head()




,label,text,clean_text,hate_type
0,0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,rtmayasolovelyasawomanyoushouldntcomplainaboutcleaningupyourhouseampasamanyoushouldalwaystakethetrashout,non_hateful
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,rtmleewboydatscoldtygadwnbadforcuffindathoeinthestplace,non_hateful
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,rturkindofbranddawgrtsbabylifeyoueverfuckabitchandshestarttocryyoubeconfusedasshit,non_hateful
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,rtcgandersonvivabasedshelooklikeatranny,non_hateful
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,rtshenikarobertstheshityouhearaboutmemightbetrueoritmightbefakerthanthebitchwhotoldittoya,non_hateful


In [ ]:
# Ensure df exists and has required cols
assert 'text' in df.columns and 'label' in df.columns and 'hate_type' in df.columns, "df must have columns: text,label,hate_type"

# Keep original index in a column so we can merge predictions exactly later
df = df.reset_index().rename(columns={'index':'orig_index'})  # orig_index == original integer index

# Filter hateful rows
df_hate = df[df['label'] == 1].copy()
n_hate = len(df_hate)
print("Total hateful rows:", n_hate)

# Sample for teacher labeling (if dataset smaller than SAMPLE_SIZE, take all)
sample_size = min(SAMPLE_SIZE, n_hate)
df_sample = df_hate.sample(n=sample_size, random_state=42).reset_index(drop=True)
print("Sampled rows for teacher labeling:", len(df_sample))

# Save sample to disk (so you don't lose it if session restarts)
df_sample.to_csv("hate_sample_for_labeling.csv", index=False)
print("Saved sample -> hate_sample_for_labeling.csv")


Total hateful rows: 233289
Sampled rows for teacher labeling: 3000
Saved sample -> hate_sample_for_labeling.csv


In [ ]:
from transformers import pipeline

# load zero-shot pipeline (BART-MNLI)
classifier = pipeline("zero-shot-classification", model=ZERO_SHOT_MODEL, device=device)

def label_batch(texts):
    """
    Send a list of texts to the zero-shot pipeline and return (predicted_label, score) for each text.
    """
    out = classifier(texts, CANDIDATE_LABELS, truncation=True)
    # out is list of dicts; each dict contains 'labels' (ranked) and 'scores'
    preds = [r['labels'][0] for r in out]
    scores = [r['scores'][0] for r in out]
    return preds, scores

# Run teacher labeling in batches and attach to df_sample
preds_all = []
scores_all = []

for i in tqdm(range(0, len(df_sample), ZERO_SHOT_BATCH), desc="Zero-shot labeling"):
    batch_texts = df_sample['text'].iloc[i:i+ZERO_SHOT_BATCH].astype(str).tolist()  # use original 'text' with spaces
    preds, scores = label_batch(batch_texts)
    preds_all.extend(preds)
    scores_all.extend(scores)

df_sample['hate_type_teacher'] = preds_all
df_sample['teacher_score'] = scores_all

# quick check
print(df_sample['hate_type_teacher'].value_counts())
df_sample.to_csv("hate_sample_labeled_by_teacher.csv", index=False)
print("Saved -> hate_sample_labeled_by_teacher.csv")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Zero-shot labeling:   0%|          | 0/94 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


hate_type_teacher
sarcasm-based hate        1687
metaphor-based hate        584
meme-language hate         350
political framing hate     192
humor-based hate           187
Name: count, dtype: int64
Saved -> hate_sample_labeled_by_teacher.csv


In [ ]:
# Show a few examples per class for manual inspection
for lab in CANDIDATE_LABELS:
    subset = df_sample[df_sample['hate_type_teacher'] == lab]
    print(f"\n=== SAMPLE for: {lab} (n={len(subset)}) ===")
    display(subset[['text','teacher_score']].sample(min(5, len(subset)), random_state=42))

# OPTIONAL: filter out very low confidence teacher labels (if you want)
CONF_THRESH = 0.35   # you can raise this to keep only confident teacher labels
print("Keeping teacher labels with score >=", CONF_THRESH)
df_sample_ft = df_sample[df_sample['teacher_score'] >= CONF_THRESH].reset_index(drop=True)
print("After filtering by score, training rows:", len(df_sample_ft))
# if filtering too much, reduce CONF_THRESH
df_sample_ft.to_csv("hate_sample_for_finetune.csv", index=False)



=== SAMPLE for: sarcasm-based hate (n=1687) ===


,text,teacher_score
580,Stop being a nigger about it.,0.522402
2814,"@HorribleLilMe Well it's a very retarded and irresponsible way to approach this issue, that can get many people killed. If you don't know how to engage in conversation and debate like a civilized person you should just shut up.",0.388226
358,@nicoleee231_ @siairamist @coldestwinters Bitch I'm just saying it's foolish to be mad over something that most ppl aren't even privy to. Fuck y'all ignorant ass ghetto bitches stay in the gutter for all I care & I give out ass beatings so PUT SOME RESPEKT ON IT BITCH. I came at y'all wit only respect but I get grimey 2,0.689547
1130,@user This is the realest (and tealest) shit ever. Eric da gawd,0.576931
926,RT @ResidentShrink: @JayInslee It is alt-left idiots like him who need to be incarcerated for breaking our nation's laws. https://t.co/5HTL…,0.329638



=== SAMPLE for: humor-based hate (n=187) ===


,text,teacher_score
2985,"Wasn't it Louise C.K. that had a bit about how saying ""n-word"" is just as bad as just saying nigger. Because thats what everyone already hears in their heads when spoken.",0.298787
1129,The us woman's team is a fucking disgrace to footballs everywhere!!! Why do they even play of they hate there own nation??? Fucking American's lol,0.454447
806,RT @NathanDuffy2: @YesYoureSexist @DangeGroth Women AREN'T funny. Not sexist it's a FACT. Get over it.,0.497135
2227,Yeah thanks to however did that because now the stupid fish guy can get off on stupid information Wrestlinglover420,0.466710
2595,"lmao, the flat tranny titties\n\nthis is a perfectly ripened shitpost",0.396902



=== SAMPLE for: metaphor-based hate (n=584) ===


...


=== SAMPLE for: political framing hate (n=192) ===


,text,teacher_score
670,"@user @user @user Well, nobody seems to be doing anything. People should be ""standing up"" by now. People should NOT take anything out on the immigrants. They have been given a free pass by the People &amp; Elites who control t",0.310492
2276,RT @BenCohenOpinion: Neo-Nazis and Holocaust deniers adore @MaxBlumenthal because he thinks just like them. More @tabletmag http://t.co/bnO…,0.395639
1262,"NEWLINE_TOKEN::Yes, because D and his mob are out to ruin everyting to do with the Larkin article and are also extremely rude and insulting and should be blocked.",0.505764
2371,@PatriotessWings @NancyPelosi The Democrats don't want to fix immigration because they look at immigrants as Democratic votes. The Republicans don't want to fix immigration because #1 They are spineless #2 They are afraid that the communist #DNC will accuse them of being heartless. #SendThemBack,0.510772
1990,hey cupcake shooter terrorist extremists leftist anti trumper... anti leo thoughts noted. th @url,0.259691



=== SAMPLE for: meme-language hate (n=350) ===


,text,teacher_score
1455,Ben Gaelys Gay \n\nBen Gaelys Gay,0.414545
2950,I fucking hate the way black women look,0.525421
2666,I love how hateful kykes are,0.444091
2028,Indians are a pack of dirty filthy cunts When they're not raping kids they're shitting in the street . Let the cunts swim in the Ganges hopefully they'll all die of some disease.,0.317319
1425,meh more likely it the digital version of rent seeking racist kike slumlords son do not rent to white men white men insist that the plumbing works and do not rent to your brother he deserves better than a ghetto,0.383447


Keeping teacher labels with score >= 0.35
After filtering by score, training rows: 1988
